### Example notebook to inspect a set of targetids with Prospect

E. Armengaud - A. Raichoor


In [1]:
# Load modules :
# - use DESI master kernel
# - get prospect from desihub : https://github.com/desihub/prospect/ (master branch)
# - setup pythonpath, eg :
import sys, os, glob
# If not using the desiconda version of prospect: EDIT THIS to your path
#sys.path.insert(0,"/global/homes/X/XXXXX/prospect/py") 
sys.path.insert(0,"/global/homes/a/armengau/prospect/py")
from prospect import plotframes, utilities
import desispec.io
from astropy.table import Table, vstack
import numpy as np
from astropy.io import fits

#### Example 1)
- minisv2 tile-based data with night-based coadds ("daily" data reduction, March 2020)
- prospect tag >= 0.1.0 (doesn't use redrock hdf5 files)


In [2]:
# Create "homemade mini-db"
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/' # EDIT THIS
tiles = ['70002', '70003', '70005'] # EDIT THIS
petals = ['0','3','6','7','9'] # EDIT THIS
obs_db = utilities.make_targetdict(tiledir, petals=petals, tiles=tiles) # petals, tiles = optional arguments

## Enter list of targets here. Targetids must be int64.
# In this example we select some random targets from the minidb
targets = [ obs_db['70002-20200304']['targetid'][34], obs_db['70005-20200303']['targetid'][2] ]
# Other example : list of SDSS stars classified as galaxies 
targets = [ 35191248222424683, 35191274004810087, 35191255705059574]

## Make spectra+zcat
# spectra and zcat are entry-matched
# there are several entries per targetid if a given target is observed several nights
spectra, zcat= utilities.load_spectra_zcat_from_targets(targets, tiledir, obs_db)

# VI interface in notebook
plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET')

#### Example 2)
- andes data reduction
- requires prospect tag >= 0.2.1 (use redrock hdf5 files)

In [ ]:
# Create "homemade mini-db"
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/' # EDIT THIS
tiles = ['68000','68001','68002'] # EDIT THIS
nights = ['20200315','20200315'] # EDIT THIS
obs_db = utilities.make_targetdict(tiledir, tiles=tiles, nights=nights) # petals, tiles, nights = optional arguments

## Enter list of targets here. Targetids must be int64.
targets = [ 35191281135124504, 35191295232180275]

## Make spectra+zcat
# spectra and zcat are entry-matched
# there are several entries per targetid if a given target is observed several nights
spectra, zcat, rrtable = utilities.load_spectra_zcat_from_targets(targets, tiledir, obs_db, with_redrock=True)

# VI interface in notebook
plotframes.plotspectra(spectra, zcatalog=zcat, redrock_cat=rrtable, title='Target_select', notebook=True, mask_type='CMX_TARGET')